In [ ]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import sqlite3
from paddleocr import PaddleOCR
import os

Correção de Rotação (Deskew)

In [ ]:

def deskew(image: np.ndarray) -> np.ndarray:
    """Corrige a inclinação de uma imagem."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray) # Inverte preto com branco
    
    # Encontra as coordenadas dos pixels que não são pretos
    coords = np.column_stack(np.where(gray > 0))
    
    # minAreaRect encontra o menor retângulo (possivelmente rotacionado)
    # que contém todos os pontos
    angle = cv2.minAreaRect(coords)[-1]

    # O ângulo retornado pode precisar de ajuste
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    # Rotaciona a imagem para corrigir a inclinação
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), 
                             flags=cv2.INTER_CUBIC, 
                             borderMode=cv2.BORDER_REPLICATE)
    
    print(f"[INFO] Ângulo de inclinação detectado: {angle:.4f} graus")
    return rotated

Verificação e Ajuste de Resolução (DPI)

In [ ]:
#ja feito no service docs

Remoção de Ruído (Denoising)

In [ ]:
def remove_noise(image: np.ndarray) -> np.ndarray:
    """Aplica um filtro de mediana para remover ruído."""
    # O segundo argumento é o tamanho do kernel. 
    # Deve ser um número ímpar (3, 5, etc.).
    return cv2.medianBlur(image, 3)

Binarização e Limiarização (Thresholding)

In [ ]:
def binarize(image: np.ndarray) -> np.ndarray:
    """Converte a imagem para preto e branco usando limiarização adaptativa."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Aplica a limiarização adaptativa
    # cv2.ADAPTIVE_THRESH_GAUSSIAN_C: método de cálculo do limiar
    # cv2.THRESH_BINARY: tipo de limiarização
    # 11: tamanho do bloco de vizinhança
    # 2: constante subtraída da média
    binary = cv2.adaptiveThreshold(gray, 255, 
                                   cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                   cv2.THRESH_BINARY, 11, 2)
    return binary

BANCO DE DADOS E LÓGICA DE PROCESSAMENTO

In [ ]:
def setup_database(db_name="document_ocr.db"):
    """Cria a tabela no banco de dados SQLite se ela não existir."""
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS ocr_results (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        pdf_name TEXT NOT NULL,
        page_number INTEGER NOT NULL,
        full_text TEXT,
        processed_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)
    conn.commit()
    return conn

In [ ]:
def convert_page_to_image(page: fitz.Page) -> np.ndarray:
    """Converte uma página de PDF em uma imagem OpenCV."""
    # Renderiza a página com 300 DPI, ideal para OCR
    pix = page.get_pixmap(dpi=300)
    # Converte os bytes da imagem para um array numpy
    img = np.frombuffer(pix.samples, dtype=np.uint8).reshape(pix.h, pix.w, pix.n)
    # Converte de RGB para BGR (padrão do OpenCV)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    return img_bgr

In [ ]:
def extract_text_from_image(ocr_agent, image: np.ndarray) -> str:
    """Usa o PaddleOCR para extrair texto de uma imagem."""
    result = ocr_agent.ocr(image)
    
    if result and result[0] is not None:
        text_lines = [line[1][0] for line in result[0]]
        return "\n".join(text_lines)
    return ""

MAIN

In [ ]:
def main(pdf_path: str):
    """
    Pipeline completo: abre um PDF, processa cada página e salva o resultado no DB.
    """
    if not os.path.exists(pdf_path):
        print(f"Erro: Arquivo PDF não encontrado em '{pdf_path}'")
        return

    pdf_name = os.path.basename(pdf_path)
    print(f"Iniciando processamento do arquivo: {pdf_name}")

    # --- Inicialização ---
    db_connection = setup_database()
    # Inicializa o PaddleOCR para português. A primeira execução fará o download dos modelos.
    print("Inicializando o PaddleOCR (pode levar um tempo na primeira vez)...")
    ocr = PaddleOCR(use_angle_cls=True, lang='pt')
    
    # --- Processamento do PDF ---
    doc = fitz.open(pdf_path)
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        print(f"\n--- Processando Página {page_num + 1}/{len(doc)} ---")

        # 1. Converter PDF -> Imagem
        print("1. Convertendo página para imagem...")
        original_image = convert_page_to_image(page)

        # 2. Aplicar pipeline de pré-processamento
        print("2. Aplicando pré-processamento de imagem...")
        cv2.imwrite(f"debug_nochange_{page_num + 1}.png", original_image)
        
        deskewed_image = deskew(original_image)
        cv2.imwrite(f"debug_deskew_{page_num + 1}.png", deskewed_image)
        
        denoised_image = remove_noise(deskewed_image)
        cv2.imwrite(f"debug_denoise_{page_num + 1}.png", denoised_image)
        
        binary_image = binarize(denoised_image)
        cv2.imwrite(f"debug_binary_{page_num + 1}.png", binary_image)
        
        # Converte a imagem binária (1 canal) de volta para BGR (3 canais)
        image_for_ocr = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2BGR)

        # 3. Executar OCR na imagem limpa
        print("3. Executando OCR com Paddle...")
        # Envia a imagem no formato correto para o OCR
        extracted_text = extract_text_from_image(ocr, image_for_ocr)
        
        if not extracted_text:
            print("Nenhum texto detectado nesta página.")
            continue
            
        # print(f"Texto extraído:\n{extracted_text[:200]}...") # Mostra um trecho

        # 4. Salvar no banco de dados
        print("4. Salvando resultado no banco de dados SQLite...")
        cursor = db_connection.cursor()
        cursor.execute(
            "INSERT INTO ocr_results (pdf_name, page_number, full_text) VALUES (?, ?, ?)",
            (pdf_name, page_num + 1, extracted_text)
        )
        db_connection.commit()
        print("Resultado salvo com sucesso!")

    # --- Finalização ---
    db_connection.close()
    doc.close()
    print(f"\nProcessamento de '{pdf_name}' concluído com sucesso!")


if __name__ == '__main__':
    # Coloque o caminho para o seu PDF de exemplo aqui
    caminho_do_pdf = "1.pdf" 
    main(caminho_do_pdf)